In [1]:
# -*- coding: utf-8 -*-

In [55]:
import pickle
import numpy as np
from PIL import Image
import paddle.v2 as paddle


In [3]:
def open_image(img_path):
    img = Image.open(img_path)
    img = np.array(img)
    return img

## Processing the language

In [4]:
token_file_path = 'data/Flickr8k_text/Flickr8k.token.txt'

with open(token_file_path, 'r') as f:
    lines = f.readlines()
lines = [x.strip('\n\r') for x in lines]

In [5]:
sentences = dict()
for line in lines:
    elements = line.split('\t')
    img_name = elements[0].split('#')[0]
    if img_name not in sentences:
        sentences[img_name] = elements[1]

In [53]:
print(sentences.keys()[1:10])

['3132619510_7dfc947d25.jpg', '3074842262_62b1b2168c.jpg', '3106782647_b078830a9e.jpg', '3447876218_4ccf42d7a0.jpg', '2966552760_e65b22cd26.jpg', '3523471597_87e0bf3b21.jpg', '3538213870_9856a76b2a.jpg', '2824401212_8da8ab99d6.jpg', '3655773435_c234e94820.jpg']


In [7]:
sentence_words = dict()
def build_dict():
    tokens = dict()
    for key in sentences.keys():
        words = sentences[key].lower().split(' ')
        if words[-1] == '.':
            words = words[:-1]
        sentence_words[key] = words
        for word in words:
            num = tokens.get(word, 0)
            tokens[word] = num + 1
    return tokens

tokens = build_dict()

In [8]:
__UNK__ = 'UNK'
__GO__ = 'GO'
__EOS__ = 'EOS'

In [9]:
vocabulary = [__UNK__, __GO__, __EOS__] + sorted(tokens, key=tokens.get, reverse=True)

In [10]:
print(vocabulary[:20])

['UNK', 'GO', 'EOS', 'a', 'in', 'the', 'and', 'on', 'is', 'dog', 'of', 'man', 'with', 'black', 'boy', 'white', 'girl', 'brown', 'are', 'to']


In [38]:
word_id = {}
id_word = {}
for (i, v) in enumerate(vocabulary):
    word_id[v] = i
    id_word[i] = v

In [39]:
dict_dim = len(word_id)
print(dict_dim)

4529


In [40]:
# Convert sentence to integers
def convert2id(words_ids, sentences):
    result = {}
    for key in sentences.keys():
        
        words = sentences[key]
        word_id = [words_ids.get(x, 0) for x in words ]
        result[key] = word_id
    
    return result

In [41]:
sentences_ids = convert2id(word_id, sentence_words)

In [42]:
print(sentences[sentences_ids.keys()[1]], sentences_ids[sentences_ids.keys()[1]])

('A white dog with brown ears leaps at a white dog running from it .', [3, 15, 9, 12, 17, 531, 244, 20, 3, 15, 9, 34, 60, 132])


In [43]:
def save_dict(dict_path, word2id, id2word):
    f = open(dict_path+'word2id.pkl', 'wb')
    pickle.dump(word2id, f)
    f.close()
        
    f = open(dict_path+'id2word.pkl', 'wb')
    pickle.dump(id2word, f)
    f.close()


In [44]:
def load_dict(dict_path):
    f = open(dict_path+'word2id.pkl', 'rb')
    word2id = pickle.load(f)
    f.close()
        
    f = open(dict_path+'id2word.pkl', 'rb')
    id2word = pickle.load(f)
    f.close()
    return word2id, id2word

In [49]:
save_dict('dict/', word_id, id_word)

In [50]:
p, m = load_dict('dict/')

In [54]:
img_dir = '/Users/Vic/Dev/DeepLearning/Paddle/DeepLearningWithPaddle/ImageCaption/data/'
img_paths = [img_dir+'10815824_2997e03d76.jpg', img_dir+'58363930_0544844edd.jpg']

(3, 224, 224)